In [ ]:
import cv2 # Biblioteca para processamento de imagens e vídeos
import os # Módulo de manipulação do sistema operacional, permitindo a interação com o OS
import threading # Módulo para execução de multi-tarefas, permitindo rodar diferentes partes do programa simultaneamente, sem que uma bloqueie a outra.
import tkinter as tk # Biblioteca para criação de interfaces gráficas - GUI
from tkinter import ttk # Módulo do Tkinter que fornece widgets mais modernos
from tkinter import filedialog # Modulo utilizado para selecionar arquivos e pastas dentro de uma GUI

In [ ]:
# Classe responsável pelo conjunto de funções e operações do player de vídeo

class VideoPlayer:

    def show_controls(self):

        control_window = tk.Tk() # Cria uma nova janela principal do tkinter, inicializando a GUI
        control_window.title("Controles do Player") # Define o título da janela criada
        control_window.geometry("400x600") # Define o tamanho da tela criada
        control_window.resizable(False, False) # Impede o redimensionamento de tela, tanto na horizontal como na vertical

        # É criado um dicionário para criar automaticamente os botões e a descrição na GUI
        # As chaves representam os títulos das seções (exemplo: "Controles de Reprodução").
        # Os valores são listas contendo os comandos correspondentes a cada seção.

        # Obervação - explicação de dicionário:
        # Um dicionário é uma estrutura de dados em Python que armazena valores associados a chaves únicas. 
        # Ele funciona como uma tabela de associação, onde cada chave tem um valor correspondente, dentre suas
        # principais características, temos:
        
        # Utiliza pares chave-valor (chave: valor).
        # As chaves devem ser únicas (não podem se repetir).
        # Os valores podem ser de qualquer tipo (inteiros, listas, strings, até outros dicionários).
        # Os dicionários são mutáveis (podem ser modificados).

        sections = {
            "Controles de Reprodução": [
                "SPACE - Play/Pause",
                "Q - Sair",
            ],
            "Navegação de Frames": [
                "A - Retroceder 1 frame (Pause)",
                "D - Avançar 1 frame (Pause)"
            ],
            "Velocidade de Reprodução": [
                "1-5 - Ajustar velocidade"
            ],
            "Captura de Frames": [
                "J - Salvar como 'Indolor'",
                "K - Salvar como 'Pouca dor'",
                "L - Salvar como 'Muita dor'"
            ]
        }

        # Percorre cada uma das chaves e valores do dicionário 'sections'
        # As chaves (section) se referem aos títulos das seções (ex. "Controles de reprodução")
        # Os valores (commands) se referem à lista de comandos daquela seção
        # A função section.items() retorna a chave e os valores simultaneamente
        for section, commands in sections.items():
            
            # É criado um grupo de controles dentro da janela principal (control_window)
            # O grupo de controles (LabelFrame) terá o títula da section e um espaçamento interno de 10px (padding)
            # O grupo de controle é armazenado na variável frame
            frame = ttk.LabelFrame(control_window, text=section, padding=10) 

            # A função pack() no Tkinter é um gerenciador de geometria responsável por posicionar os widgets (componentes gráficos) dentro da janela. 
            # Ele organiza os elementos automaticamente de forma sequencial (vertical ou horizontal), sem precisar definir coordenadas específicas.
            # Quando chamamos widget.pack(), o Tkinter coloca esse widget dentro do container pai (geralmente a janela principal ou um Frame). 
            # Ele segue uma lógica de empilhamento.

            #  fill="both": Faz com que o frame ocupe todo o espaço disponível na horizontal.
            # expand=True: Permite que o frame cresça caso a janela seja expandida.
            # padx=10, pady=5: Adiciona margens externas de 10 pixels na horizontal e 5 pixels na vertical. 
            frame.pack(fill="both", expand=True, padx=10, pady=5)

            for command in commands: # Percorre a lista de valores dentre da seção atual

                # É criado um rótulo (label) para exibir o texto do comando dentro do grupo de controle (frame)
                # text=command: Define o texto do rótulo como o comando atual.
                # font=("Arial", 10): Define a fonte do texto como Arial tamanho 10.
                # anchor="w": Alinha o texto à esquerda.
                # justify="left": Garante que o texto fique alinhado corretamente.
                label = tk.Label(frame, text=command, font=("Arial", 10), anchor="w", justify="left")

                # anchor="w": Mantém o alinhamento à esquerda dentro do frame.
                # padx=10, pady=2: Adiciona espaçamento horizontal e vertical ao rótulo.
                label.pack(anchor="w", padx=10, pady=2)

        # É criado um botão na janela principal para fechar a tela principal caso seja clicado
        # O que é definido por command=control_window.destroy  
        close_button = tk.Button(control_window, text="Fechar", command=control_window.destroy)
        # Adiciona um espaçamento vertical de 10 pixels ao botão.
        close_button.pack(pady=10)

        # Mantém a janela aberta até que o usuário a feche manualmente.
        # Esse é o loop principal do Tkinter, que mantém a interface responsiva.
        control_window.mainloop()


    # Método especial responsável por inicializar os atributos do objeto
    # sendo executado automáticamente ao criar um novo objeto da classe.
    # Em Python o método CONSTRUTOR é definido como __init__

    # O self trata-se de uma referência ao próprio objeto.
    # É importante entender que cada objeto criado a partir 
    # de uma classe tem seus próprios atributos, com isso, 
    # o self garante que os métodos da classe possam acessar 
    # e modificar tais atributos.

    def __init__(self, video_path, fps=30):
        
        self.video_path = video_path # Passa o caminho do objeto
        self.fps = fps # Target FPS
        self.cap = cv2.VideoCapture(self.video_path) # Carrega o arquivo de vídeo

        # Valida se o arquivo de vídeo for aberto corretamente, sendo cap o arquivo de vídeo carregado pelo cv2
        if not self.cap.isOpened():
            raise ValueError("Erro ao abrir o vídeo.")
        
        self.frame_delay = int(1000 / self.fps) # Tempo de espera entre frames em milissegundos
        self.paused = True # Flag para controlar o pause
        self.frame_index = 0 #Denota a posição inicial do frame
        self.playback_speed = 1  # Velocidade de reprodução padrão
        self.current_frame = None  # Armazena o frame atual

        # É criado um dicionário para definir as pastas que serão criadas para o armazenamento dos frames
        # O dicionário é então percorrido, realizando a leitura APENAS dos valores e ignorando as chaves (self.output_dirs.values())
        # São criados diretórios com o mesmo nome do valor dos dicionários
        self.output_dirs = {"J": "Indolor", "K": "Pouca dor", "L": "Muita dor"}
        for directory in self.output_dirs.values():
            os.makedirs(directory, exist_ok=True)

        self.display_message = ""  # Mensagem de salvamento a ser exibida no vídeo
        self.message_duration = 3  # Quantidade de frames que a mensagem será exibida
        self.message_counter = 0

    # Essa função atualiza o frame do vídeo e exibe a imagem atualizada na janela do OpenCV sempre que houver 
    # alguma modificação, como mensagem na tela ou passagem manual de frames.
    def update_frame(self):

        # Posiciona o vídeo exatamente no frame self.frame_index, garantindo que exibiremos a imagem correta.
        # isso é útil uma vez que nem sempre o vídeo será exibido de maneira linear
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, self.frame_index)
        
        # Realiza a leitura do frame, retornando um booleano para informar se a leitura foi bem sucedida e 
        # o frame lido em formato de uma matriz numpy
        ret, frame = self.cap.read() 

        # Caso a leitura do frame tenha sido bem sucedida, temos:
        if ret:

            self.current_frame = frame # armazena a imagem lida (frame) na variável self.current_frame.
            self._draw_info(frame) # o frame é passado como argumento para a função _draw_info a fim de exibir as informações de controle do frame
            cv2.imshow('Video Player', frame) # O frame lido é exibido para o usuário

    # Função responsável pelo salvamento dos frames em alguma das pastas definidas em 'output_dirs'

    def save_frame(self, category):

        if self.current_frame is not None: # Valida o ultimo frame lido, evitando salvar um frame vazio ou inexistente
            
            # folder recebe o nome da pasta contida no dicionário output_dirs (criado préviamente) definido
            # por sua categoria (j, k ou l) 
            folder = self.output_dirs.get(category) 
            
            # Valida se a tecla pressionada (o parâmetro category) contém uma entrada no dicionário
            if folder:

                filename = os.path.join(folder, f"frame_{self.frame_index}.png") # cria o caminho completo para o arquivo, incluindo o nome para o frame
                cv2.imwrite(filename, self.current_frame) # Salva o arquivo, sendo o primeiro argumento o caminho completo e o segundo argumento o frame
                self.display_message = f"Frame {self.frame_index} salvo ({category})" # Gera a mensagem a ser exibida para o usuário sobre o status da operação
                self.message_counter = self.message_duration # Define o tempo que a mensagem deve ficar visível
                print(f"Frame salvo em {filename}") # Mensagem de log sobre o status da operação
        else:

            print("Erro: Nenhum frame carregado para salvar.") # Mensagem de erro caso o frame lido não seja valido ou existente

    # Essa função tem a finalidade de desenhar informações na tela do vídeo enquanto ele está sendo exibido. 

    def _draw_info(self, frame):

        # É inserido um texto no frame, neste caso o valor de posição do frame é exibido
    
        # cv2.putText() -> adiciona um texto na imagem (frame).
        # O segundo parâmetro indica o que será escrito na tela
        # (10, 30) -> Coordenadas (x, y) do canto esquerdo superior onde o texto será colocado.
        # cv2.FONT_HERSHEY_SIMPLEX -> Define a fonte do texto.
        # 1 -> Define o tamanho da fonte.
        # (0, 255, 0) -> Define a cor do texto (verde no formato BGR: Azul, Verde, Vermelho).
        # 2 -> Define a espessura da linha do texto.

        cv2.putText(frame, f"Frame: {self.frame_index}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Valida se há ou não alguma mensagem de salvamento a ser exibida (de acordo com a sua duração de exibição)
        if self.message_counter > 0: 

            # Um novo texto é exibido, definido por display_message
            cv2.putText(frame, self.display_message, (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
            # O contador de tempo é decrementado, com isso a mensagem some após uma determinada
            # quantidade de frames de execução
            self.message_counter -= 1

    def play(self):
        
        threading.Thread(target=self.show_controls, daemon=True).start()
        self.update_frame()
        
        while self.cap.isOpened():
            
            if not self.paused:
                
                for _ in range(self.playback_speed):
                    
                    ret, frame = self.cap.read()
                    
                    if not ret:
                        break
                    
                    self.frame_index = int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))
                    self.current_frame = frame.copy()
                
                self._draw_info(frame)
                cv2.imshow('Video Player', frame)
            
            key = cv2.waitKey(self.frame_delay) & 0xFF

            if key == ord('q'):
                break

            elif key == ord(' '):
                self.paused = not self.paused
            
            elif self.paused and key == ord('d'):
                self.frame_index += 1
                self.update_frame()
            
            elif self.paused and key == ord('a'):
                self.frame_index = max(self.frame_index - 1, 0)
                self.update_frame()
            
            elif key in [ord(str(i)) for i in range(1, 6)]:
                self.playback_speed = int(chr(key))
            
            elif self.paused and key in [ord('j'), ord('k'), ord('l')]:
                self.save_frame(chr(key).upper())
                self.update_frame()
        
        self.cap.release()
        cv2.destroyAllWindows()
    
    def __del__(self):
        if self.cap.isOpened():
            self.cap.release()
            cv2.destroyAllWindows()

In [12]:
# O tkinter é utilizado para a criação de interfaces gráficas
# como janelas, botões, caixas de entrada, etc.
# Neste caso, ele é utilizado para criar a janela que permite
# a escolha do arquivo de vídeo.

root = tk.Tk() # Cria a instância do tkinter
root.withdraw()  # Oculta a janela principal da interface, mantendo apenas o necessário
root.attributes('-topmost', True)  # Força a janela a ficar no topo
root.update()  # Atualiza a janela para aplicar as mudanças

# Trecho responsável por abrir uma janela para a escolha de um arquivo (seu path)

video_path = filedialog.askopenfilename( # A função do tkinter abre o explorador de arquivos
    title="Selecione um vídeo", # Nome da janela a ser aberta
    filetypes=[("Arquivos de vídeo", "*.mp4;*.avi;*.mov;*.mkv")], # Define quais arquivos podem ser escolhidos
    parent=root # Define a janela oculta do tkinter (root) como a janela pai do explorador, com isso
                # as alterações em root afetam a janela do explorador (como manter ela em primeiro plano)
    )

root.destroy() # Elimina a janela após a escolha do arquivo

# Exibe ao usuário do programa se algum arquivo foi ou não escolhido
# de acordo com a presença de um caminho na variável video_path   

if not video_path:
    print("Nenhum arquivo selecionado.")
else:
    print(f"Arquivo selecionado: {video_path}")

Arquivo selecionado: C:/Users/marci_wawp/Desktop/Arquivos/Mestrado/PRJ_MestradoV2/Videos/WhatsApp Video 2025-01-28 at 13.50.50.mp4


In [13]:

player = VideoPlayer(video_path, fps=30)
player.play()


Frame salvo em Indolor\frame_15.png
Frame salvo em Pouca dor\frame_25.png
Frame salvo em Muita dor\frame_40.png
Frame salvo em Indolor\frame_41.png
Frame salvo em Pouca dor\frame_42.png
